# Load rankings and categories

In [13]:
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import json,os,pdb
import tqdm
import warnings

# Choose the universe

Select the number of coins to consider. Right now we are still using a market cap criteria

In [14]:
N = 50

# Load data


In [15]:
from pandas import read_parquet
#
data = read_parquet("hourly.parquet.gzip")
display(data.head())
# print(data.index.values)

time_open  \
symbol_id time_close                                           
ZIL_2469  2019-01-01T20:59:59.999Z  2019-01-01T20:00:00.000Z   
          2019-01-01T21:59:59.999Z  2019-01-01T21:00:00.000Z   
          2019-01-01T22:59:59.999Z  2019-01-01T22:00:00.000Z   
          2019-01-01T23:59:59.999Z  2019-01-01T23:00:00.000Z   
          2019-01-02T00:59:59.999Z  2019-01-02T00:00:00.000Z   

                                                   time_high  \
symbol_id time_close                                           
ZIL_2469  2019-01-01T20:59:59.999Z  2019-01-01T20:52:19.000Z   
          2019-01-01T21:59:59.999Z  2019-01-01T21:17:18.000Z   
          2019-01-01T22:59:59.999Z  2019-01-01T22:45:18.000Z   
          2019-01-01T23:59:59.999Z  2019-01-01T23:31:17.000Z   
          2019-01-02T00:59:59.999Z  2019-01-02T00:04:21.000Z   

                                                    time_low  quote.USD.open  \
symbol_id time_close                                                           
ZIL_2469  2019-01-01T20:59:59.999Z  2019-01-01T20:00:15.000Z        0.019861   
          2019-01-01T21:59:59.999Z  2019-01-01T21:01:21.000Z        0.020221   
          2019-01-01T22:59:59.999Z  2019-01-01T22:23:19.000Z        0.020231   
          2019-01-01T23:59:59.999Z  2019-01-01T23:00:16.000Z        0.020278   
          2019-01-02T00:59:59.999Z  2019-01-02T00:45:17.000Z        0.020800   

                                    quote.USD.high  quote.USD.low  \
symbol_id time_close                                                
ZIL_2469  2019-01-01T20:59:59.999Z        0.020177       0.019861   
          2019-01-01T21:59:59.999Z        0.020316       0.020152   
          2019-01-01T22:59:59.999Z        0.020285       0.020080   
          2019-01-01T23:59:59.999Z        0.020830       0.020269   
          2019-01-02T00:59:59.999Z        0.020800       0.020442   

                                    quote.USD.close  quote.USD.market_cap  \
symbol_id time_close                                                        
ZIL_2469  2019-01-01T20:59:59.999Z         0.020133          1.878770e+08   
          2019-01-01T21:59:59.999Z         0.020208          1.885781e+08   
          2019-01-01T22:59:59.999Z         0.020236          1.888430e+08   
          2019-01-01T23:59:59.999Z         0.020717          1.933268e+08   
          2019-01-02T00:59:59.999Z         0.020528          1.915641e+08   

                                    quote.USD.volume  \
symbol_id time_close                                   
ZIL_2469  2019-01-01T20:59:59.999Z               NaN   
          2019-01-01T21:59:59.999Z               NaN   
          2019-01-01T22:59:59.999Z               NaN   
          2019-01-01T23:59:59.999Z               NaN   
          2019-01-02T00:59:59.999Z               NaN   

                                         quote.USD.timestamp symbol    id  
symbol_id time_close                                                       
ZIL_2469  2019-01-01T20:59:59.999Z  2019-01-01T20:59:59.999Z    ZIL  2469  
          2019-01-01T21:59:59.999Z  2019-01-01T21:59:59.999Z    ZIL  2469  
          2019-01-01T22:59:59.999Z  2019-01-01T22:59:59.999Z    ZIL  2469  
          2019-01-01T23:59:59.999Z  2019-01-01T23:59:59.999Z    ZIL  2469  
          2019-01-02T00:59:59.999Z  2019-01-02T00:59:59.999Z    ZIL  2469

In [16]:
df = pd.read_parquet('hourly.parquet.gzip') # modify the path according to your needs
df.head()


time_open  \
symbol_id time_close                                           
ZIL_2469  2019-01-01T20:59:59.999Z  2019-01-01T20:00:00.000Z   
          2019-01-01T21:59:59.999Z  2019-01-01T21:00:00.000Z   
          2019-01-01T22:59:59.999Z  2019-01-01T22:00:00.000Z   
          2019-01-01T23:59:59.999Z  2019-01-01T23:00:00.000Z   
          2019-01-02T00:59:59.999Z  2019-01-02T00:00:00.000Z   

                                                   time_high  \
symbol_id time_close                                           
ZIL_2469  2019-01-01T20:59:59.999Z  2019-01-01T20:52:19.000Z   
          2019-01-01T21:59:59.999Z  2019-01-01T21:17:18.000Z   
          2019-01-01T22:59:59.999Z  2019-01-01T22:45:18.000Z   
          2019-01-01T23:59:59.999Z  2019-01-01T23:31:17.000Z   
          2019-01-02T00:59:59.999Z  2019-01-02T00:04:21.000Z   

                                                    time_low  quote.USD.open  \
symbol_id time_close                                                           
ZIL_2469  2019-01-01T20:59:59.999Z  2019-01-01T20:00:15.000Z        0.019861   
          2019-01-01T21:59:59.999Z  2019-01-01T21:01:21.000Z        0.020221   
          2019-01-01T22:59:59.999Z  2019-01-01T22:23:19.000Z        0.020231   
          2019-01-01T23:59:59.999Z  2019-01-01T23:00:16.000Z        0.020278   
          2019-01-02T00:59:59.999Z  2019-01-02T00:45:17.000Z        0.020800   

                                    quote.USD.high  quote.USD.low  \
symbol_id time_close                                                
ZIL_2469  2019-01-01T20:59:59.999Z        0.020177       0.019861   
          2019-01-01T21:59:59.999Z        0.020316       0.020152   
          2019-01-01T22:59:59.999Z        0.020285       0.020080   
          2019-01-01T23:59:59.999Z        0.020830       0.020269   
          2019-01-02T00:59:59.999Z        0.020800       0.020442   

                                    quote.USD.close  quote.USD.market_cap  \
symbol_id time_close                                                        
ZIL_2469  2019-01-01T20:59:59.999Z         0.020133          1.878770e+08   
          2019-01-01T21:59:59.999Z         0.020208          1.885781e+08   
          2019-01-01T22:59:59.999Z         0.020236          1.888430e+08   
          2019-01-01T23:59:59.999Z         0.020717          1.933268e+08   
          2019-01-02T00:59:59.999Z         0.020528          1.915641e+08   

                                    quote.USD.volume  \
symbol_id time_close                                   
ZIL_2469  2019-01-01T20:59:59.999Z               NaN   
          2019-01-01T21:59:59.999Z               NaN   
          2019-01-01T22:59:59.999Z               NaN   
          2019-01-01T23:59:59.999Z               NaN   
          2019-01-02T00:59:59.999Z               NaN   

                                         quote.USD.timestamp symbol    id  
symbol_id time_close                                                       
ZIL_2469  2019-01-01T20:59:59.999Z  2019-01-01T20:59:59.999Z    ZIL  2469  
          2019-01-01T21:59:59.999Z  2019-01-01T21:59:59.999Z    ZIL  2469  
          2019-01-01T22:59:59.999Z  2019-01-01T22:59:59.999Z    ZIL  2469  
          2019-01-01T23:59:59.999Z  2019-01-01T23:59:59.999Z    ZIL  2469  
          2019-01-02T00:59:59.999Z  2019-01-02T00:59:59.999Z    ZIL  2469

In [17]:
new_df = df[df["symbol"].str.contains('ETH')]
new_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 31676 entries, ('ETH_1027', '2019-01-01T20:59:59.999Z') to ('ETH_1027', '2022-10-10T19:59:59.999Z')
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   time_open             31676 non-null  object 
 1   time_high             31675 non-null  object 
 2   time_low              31675 non-null  object 
 3   quote.USD.open        31676 non-null  float64
 4   quote.USD.high        31676 non-null  float64
 5   quote.USD.low         31676 non-null  float64
 6   quote.USD.close       31676 non-null  float64
 7   quote.USD.market_cap  31676 non-null  float64
 8   quote.USD.volume      16526 non-null  float64
 9   quote.USD.timestamp   31676 non-null  object 
 10  symbol                31676 non-null  object 
 11  id                    31676 non-null  int64  
dtypes: float64(6), int64(1), object(5)
memory usage: 4.2+ MB


In [18]:
new_df = df[df["symbol"].str.contains('DOGE')]
new_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 31676 entries, ('DOGE_74', '2019-01-01T20:59:59.999Z') to ('DOGE_74', '2022-10-10T19:59:59.999Z')
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   time_open             31676 non-null  object 
 1   time_high             31675 non-null  object 
 2   time_low              31675 non-null  object 
 3   quote.USD.open        31676 non-null  float64
 4   quote.USD.high        31676 non-null  float64
 5   quote.USD.low         31676 non-null  float64
 6   quote.USD.close       31676 non-null  float64
 7   quote.USD.market_cap  31676 non-null  float64
 8   quote.USD.volume      16526 non-null  float64
 9   quote.USD.timestamp   31676 non-null  object 
 10  symbol                31676 non-null  object 
 11  id                    31676 non-null  int64  
dtypes: float64(6), int64(1), object(5)
memory usage: 4.2+ MB


In [19]:
with open('listing_map_20221004.json',encoding='utf-8') as json_data:
    data = json.load(json_data)
listmap = pd.DataFrame(data['data'])
listmap.sort_values('cmc_rank',inplace=True)
listmap.set_index('cmc_rank',inplace=True)
listmap['symbol_id'] = listmap['symbol']+ '_'+ listmap['id'].astype(str)
listmap.head()

,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,platform,self_reported_circulating_supply,self_reported_market_cap,tvl_ratio,last_updated,quote,symbol_id
cmc_rank,,,,,,,,,,,,,,,,,
1,1,Bitcoin,BTC,bitcoin,9760,2013-04-28T00:00:00.000Z,"[mineable, pow, sha-256, store-of-value, state...",21000000.0,1.916889e+07,1.916889e+07,None,NaN,NaN,NaN,2022-10-04T14:02:00.000Z,"{'USD': {'price': 20124.645699098404, 'volume_...",BTC_1
2,1027,Ethereum,ETH,ethereum,6121,2015-08-07T00:00:00.000Z,"[pos, smart-contracts, ethereum-ecosystem, coi...",NaN,1.226484e+08,1.226484e+08,None,NaN,NaN,NaN,2022-10-04T14:02:00.000Z,"{'USD': {'price': 1355.3241075801398, 'volume_...",ETH_1027
3,825,Tether,USDT,tether,40468,2015-02-25T00:00:00.000Z,"[payments, stablecoin, asset-backed-stablecoin...",NaN,6.794942e+10,7.015545e+10,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ET...",NaN,NaN,NaN,2022-10-04T14:02:00.000Z,"{'USD': {'price': 1.0002059770173375, 'volume_...",USDT_825
4,1839,BNB,BNB,bnb,1120,2017-07-25T00:00:00.000Z,"[marketplace, centralized-exchange, payments, ...",200000000.0,1.613373e+08,1.613373e+08,None,NaN,NaN,NaN,2022-10-04T14:02:00.000Z,"{'USD': {'price': 296.4318964135179, 'volume_2...",BNB_1839
5,3408,USD Coin,USDC,usd-coin,6429,2018-10-08T00:00:00.000Z,"[medium-of-exchange, stablecoin, asset-backed-...",NaN,4.703592e+10,4.703592e+10,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ET...",NaN,NaN,NaN,2022-10-04T14:02:00.000Z,"{'USD': {'price': 1.000153755174831, 'volume_2...",USDC_3408


# Filter by the universe
Select only the symbols that interest you. According to our previous choice of N, in this case we are filtering for the top 50 by market cap

In [20]:
symbols = list(listmap.loc[:N,'symbol_id'].values)

Remove stablecoins by using the category JSON (available in sakai as the listing file)

In [21]:
with open('category_stablecoin.json',encoding='utf-8') as json_data:
    data = json.load(json_data)
stable_df = pd.json_normalize(data['data']['coins'],max_level=5)
stable_df['symbol_id'] = stable_df['symbol'] + '_' + stable_df['id'].astype(str)
display(stable_df.head())
stablecoins = list(stable_df['symbol_id'].values)

,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,...,quote.USD.percent_change_30d,quote.USD.percent_change_60d,quote.USD.percent_change_90d,quote.USD.market_cap,quote.USD.market_cap_dominance,quote.USD.fully_diluted_market_cap,quote.USD.tvl,quote.USD.last_updated,platform,symbol_id
0,825,Tether,USDT,tether,35908,2015-02-25T00:00:00.000Z,"[payments, stablecoin, asset-backed-stablecoin...",NaN,6.599228e+10,6.816037e+10,...,0.005916,-0.051152,-0.110549,6.595395e+10,6.8691,6.812077e+10,NaN,2022-07-08T18:40:00.000Z,NaN,USDT_825
1,3408,USD Coin,USDC,usd-coin,4787,2018-10-08T00:00:00.000Z,"[medium-of-exchange, stablecoin, asset-backed-...",NaN,5.554694e+10,5.554694e+10,...,-0.040697,0.032473,0.055801,5.555146e+10,5.7844,5.555146e+10,NaN,2022-07-08T18:40:00.000Z,NaN,USDC_3408
2,4687,Binance USD,BUSD,binance-usd,4028,2019-09-20T00:00:00.000Z,"[stablecoin, asset-backed-stablecoin, binance-...",NaN,1.754722e+10,1.754722e+10,...,0.205461,0.189911,0.228809,1.758121e+10,1.8301,1.758121e+10,NaN,2022-07-08T18:41:00.000Z,NaN,BUSD_4687
3,4943,Dai,DAI,multi-collateral-dai,1019,2019-11-22T00:00:00.000Z,"[defi, stablecoin, ethereum-ecosystem, avalanc...",NaN,6.926538e+09,6.926538e+09,...,0.100228,0.269779,0.113622,6.930935e+09,0.7215,6.930935e+09,NaN,2022-07-08T18:41:00.000Z,NaN,DAI_4943
4,6952,Frax,FRAX,frax,135,2020-09-08T00:00:00.000Z,"[defi, stablecoin, algorithmic-stablecoin, sei...",NaN,1.366324e+09,1.366324e+09,...,0.260128,1.185800,0.015312,1.365766e+09,0.0000,1.365766e+09,NaN,2022-07-08T18:41:00.000Z,NaN,FRAX_6952


In [22]:
# get the top 200 by mkt cap
symbols = set(listmap.loc[:N,'symbol_id'].values) - set(stablecoins)
print(symbols)
for l in stablecoins:
    if l in symbols:
        print('Stablecoins in the list!')

{'LTC_2', 'SAND_6210', 'CRO_3635', 'LINK_1975', 'THETA_2416', 'UNI_7083', 'XTZ_2011', 'TRX_1958', 'XRP_52', 'SHIB_5994', 'FLOW_4558', 'ICP_8916', 'BTC_1', 'TON_11419', 'VET_3077', 'DOT_6636', 'EOS_1765', 'NEAR_6535', 'APE_18876', 'FTT_4195', 'OKB_3897', 'HBAR_4642', 'BNB_1839', 'MANA_1966', 'XLM_512', 'ADA_2010', 'DOGE_74', 'ATOM_3794', 'LUNC_4172', 'WBTC_3717', 'EGLD_6892', 'ALGO_4030', 'MATIC_3890', 'ETC_1321', 'QNT_3155', 'ETH_1027', 'AAVE_7278', 'XMR_328', 'AVAX_5805', 'BCH_1831', 'FIL_2280', 'SOL_5426', 'LEO_3957', 'CHZ_4066', 'AXS_6783'}
